In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pickle

# Load the preprocessed data
data = pd.read_csv('../data/trip_summary.csv')


In [4]:
# Assume that you have a 'Driving_Score' column for classification
# If not, create a dummy target variable for illustration purposes
# In practice, you should have real target data for model training
if 'Driving_Score' not in data.columns:
    # Example of creating a dummy target variable
    # Replace this with actual labels if available
    data['Driving_Score'] = (data['Speed(km/h)_mean'] > 50).astype(int)

# Define features and target variable
X = data.drop(columns=['TripId', 'Driving_Score'])
y = data['Driving_Score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Save the model
model_filename = '../models/driving_behavior_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(model, file)

print(f"Model training complete. Model saved to {model_filename}.")

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       0.86      1.00      0.92         6

    accuracy                           0.90        10
   macro avg       0.93      0.88      0.89        10
weighted avg       0.91      0.90      0.90        10

Confusion Matrix:
[[3 1]
 [0 6]]
Model training complete. Model saved to ../models/driving_behavior_model.pkl.
